In [2]:
## installing modules

In [3]:
  >>> import nltk
  >>> nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
  >>> import nltk
  >>> nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.5 MB/s eta 0:00:00


In [8]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.4 MB/s eta 0:00:00


In [9]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 17.6 MB/s eta 0:00:00


In [10]:
pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 35.1 MB/s eta 0:00:00


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
# Importing Libraries

In [13]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from contractions import fix 
from unidecode import unidecode
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [14]:
####################################################################################################

In [15]:
# First step is to convert given audio to text

In [16]:
# My first approach is to use pretrained hugging face model commonly used for speech to text conversion 
# this model is pretrained and fine-tuned on 960 hours of speech audio. 

In [17]:
# Load the pre-trained Wav2Vec model and processor
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Load and preprocess the audio file
audio, sample_rate = torchaudio.load("/content/sales_call_telephone_marketers.wav")
input_values = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_values

input_values= input_values.reshape(1, -1)
# print(input_values)

# model(input)
# Perform transcription
with torch.no_grad():
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.decode(predicted_ids[0])

# Output the transcription
print(transcription)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


HELLO IE NANCY THIS IS LIKE FROM EITIENT INCORPORATION YES HOW CAN I HELP YOU NANCY YOU HAVE BEEN USING OUR PREEP CONNETION FOR A COUPLE OF YEARS NOW RIGHT YE THAT'S RIGHT HOW WOULD YOU LIKE A POSTPA CONNECTION THAT ALLOWS YOU TO MAKE FREE UNLIMITED OIE CALLS TO THREE EIGHTENT NUMBERS I WOULD LOVE THAT BUT WHAT'S THE CATCH THERE'S NO CATCH THERE WILL BE A MONTHLY RENTAL WHICH YOU WILL HAVE TO PAY LIKE ANY OTHER POSTPAID CONNECTION FANTASTIC SIGN ME UP HELLO IE NANCY THIS IS LIKE FROM EGHTIENT INCORPORATION YES HOW CAN I HELP YOU NANCY YOU HAVE BEEN USING OUR PRE PA CONNECTION FOR A COUPLE OF YEARS NOW RIGHT YE THAT'S RIGHT HOW WOULD YOU LIKE A POSTPA CONNECTION THAT ALLOWS YOU TO MAKE FREE UNLIMITED VOICE CALLS TO THREE EITIENT NUMBERS I WOULD LOVE THAT BUT WHAT'S THE CATCH THERE'S NO CATCH THERE WILL BE A MONTHLY RENTAL WHICH YOU WILL HAVE TO PAY LIKE ANY OTHER POSTPAID CONNECTION FANTASTIC SIGN ME U


In [18]:
#As seen above the text is not correctly captured by our model
# So i thought of creating a function which will convert given audio to 16khz input and will be stored as resampled_audio and then pass through model

In [19]:
# Load the pre-trained Wav2Vec model and processor
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# ----------------------------
######## writing function to convert given audio to 16khz frequency
# Load the audio file
waveform, sample_rate = torchaudio.load("/content/sales_call_telephone_marketers.wav")

# Resample the audio to 16 kHz
resampler = torchaudio.transforms.Resample(sample_rate, 16000)
resampled_waveform = resampler(waveform)

# Save the resampled audio to a new file
torchaudio.save("resampled_audio.wav", resampled_waveform, 16000)
# ---------------------


# Load and preprocess the audio file
audio, sample_rate = torchaudio.load("/content/resampled_audio.wav")
input_values = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_values

input_values= input_values.reshape(1, -1)
# print(input_values)

# model(input)
# Perform transcription
with torch.no_grad():
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.decode(predicted_ids[0])

# Output the transcription
print(transcription)



Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


HELLO IE NANCY THIS IS LIKE FROM EITIENT INCORPORATION YES HOW CAN I HELP YOU NANCY YOU HAVE BEEN USING OUR PREEP CONNETION FOR A COUPLE OF YEARS NOW RIGHT YE THAT'S RIGHT HOW WOULD YOU LIKE A POSTPA CONNECTION THAT ALLOWS YOU TO MAKE FREE UNLIMITED OIE CALLS TO THREE EIGHTENT NUMBERS I WOULD LOVE THAT BUT WHAT'S THE CATCH THERE'S NO CATCH THERE WILL BE A MONTHLY RENTAL WHICH YOU WILL HAVE TO PAY LIKE ANY OTHER POSTPAID CONNECTION FANTASTIC SIGN ME UP HELLO IE NANCY THIS IS LIKE FROM EGHTIENT INCORPORATION YES HOW CAN I HELP YOU NANCY YOU HAVE BEEN USING OUR PRE PA CONNECTION FOR A COUPLE OF YEARS NOW RIGHT YE THAT'S RIGHT HOW WOULD YOU LIKE A POSTPA CONNECTION THAT ALLOWS YOU TO MAKE FREE UNLIMITED VOICE CALLS TO THREE EITIENT NUMBERS I WOULD LOVE THAT BUT WHAT'S THE CATCH THERE'S NO CATCH THERE WILL BE A MONTHLY RENTAL WHICH YOU WILL HAVE TO PAY LIKE ANY OTHER POSTPAID CONNECTION FANTASTIC SIGN ME U


In [20]:
# as seen above there was no change, our model is not correctly capturing the words in the audio

In [21]:
####################################################################################################

In [22]:
# I tried to get the named entity in the transcripion text using spacy library

In [23]:
import spacy
from spacy import displacy
from spacy import tokenizer

In [24]:
# loading the model for NER

In [25]:
nlp = spacy.load('en_core_web_sm')

In [26]:
# # import modules and download packages
# import nltk
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('maxent_ne_chunker')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('state_union')
# from nltk.corpus import state_union
# from nltk.tokenize import PunktSentenceTokenizer

In [27]:
nlp = spacy.load('en_core_web_sm')

In [28]:
# writing function for detecting Named Entity in the given text after passing through the model
# also wrote a function for visualizing the named entity with labels on the text

In [29]:
def print_entities(pipeline, text):
    
    # Create a document 
    document = pipeline(text)
    
    # Entity text & label extraction
    for entity in document.ents:
        print(entity.text + '->', entity.label_)
        
        
def visualize_entities(pipeline, text):
    
    # Create a document 
    document = pipeline(text)
        
    # Show entities in pretty manner
    displacy.render(document, jupyter=True, style='ent')

In [30]:
# Below are the labels and meaning of that labels given by NER model

In [31]:
# PERSON, NORP (nationalities, religious and political groups),
#  FAC (buildings, airports etc.), 
#  ORG (organizations), 
#  GPE (countries, cities etc.), 
#  LOC (mountain ranges, water bodies etc.), 
#  PRODUCT (products), 
#  EVENT (event names),
#  WORK_OF_ART (books, song titles), 
#  LAW (legal document titles), 
#  LANGUAGE (named languages), DATE, TIME, PERCENT, MONEY, QUANTITY, ORDINAL and CARDINAL.

In [32]:
print_entities(nlp, transcription)

NANCY-> ORG
EITIENT-> GPE
A COUPLE OF YEARS-> DATE
THREE-> CARDINAL
A COUPLE OF YEARS-> DATE


In [33]:
visualize_entities(nlp, transcription)

In [34]:
# We can see because of incorrect detection of words from audio, Named Entities are not properly found out.
# This approach is not giving good results 

In [35]:
####################################################################################################

In [36]:
# Second approach for converting speech to text is to use Speech Recognition library

In [37]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [38]:
#import library

import speech_recognition as sr

# #Initiаlize  reсоgnizer  сlаss  (fоr  reсоgnizing  the  sрeeсh)

r = sr.Recognizer()

# # Reading Audio file as source

# #  listening  the  аudiо  file  аnd  stоre  in  аudiо_text  vаriаble


with sr.AudioFile('/content/sales_call_telephone_marketers.wav') as source:
    audio_text = r.listen(source)

text = r.recognize_google(audio_text)

print(text)


hello Nancy this is Mike from AT&T Incorporation


In [39]:
#As we can see the result is pretty promising in this case we go forward with Sepeech Recognition model

In [40]:
# I added a try and except block to avoid error

In [41]:
def convert(audio):
  #Initiаlize  reсоgnizer  сlаss  (fоr  reсоgnizing  the  sрeeсh)
  r = sr.Recognizer()
  # Reading Audio file as source
  #  listening  the  аudiо  file  аnd  stоre  in  аudiо_text  vаriаble
  with sr.AudioFile(audio) as source:
      audio_text = r.listen(source)
  # recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
      try:
          # using google speech recognition
          text = r.recognize_google(audio_text)

          print(text)
          
      except:
          print('Sorry.. run again...')

In [42]:
convert('/content/sales_call_telephone_marketers.wav')

hello Nancy this is Mike from AT&T Incorporation


In [43]:
print(text)

hello Nancy this is Mike from AT&T Incorporation


In [44]:
####################################################################################################

In [45]:
#Now lets see the Named Entities in this text by calling the function print_entities() and visualize_entities() which is written above

In [46]:
print_entities(nlp, text)

Nancy-> PERSON
Mike-> PERSON
AT&T Incorporation-> ORG


In [47]:
visualize_entities(nlp, text)

In [48]:
# Now we can say model is correctly detecting the Person Name and organization

In [49]:
####################################################################################################

In [50]:
# We also want Intent, so I am creating a dataframe df1 and adding the examples which were provided to me in acessment pdf.

In [51]:
# This model can further be improved by providing large dataset having variety of intents

In [52]:
data={"text":["My name is Jeff and I am calling from Amazon", "I am calling from Microsoft and my name is Satya", "I am Sundar and this is call from Google",
              "I am calling about your Microsoft Azure subscription","This is a call regarding your Google Cloud Platform account", "I would like to talk about your Amazon Web Services account."],
      "Intent":["Intro", "Intro", "Intro", "Purpose", "Purpose", "Purpose"]}

df1 = pd.DataFrame(data)

df1

,text,Intent
0,My name is Jeff and I am calling from Amazon,Intro
1,I am calling from Microsoft and my name is Satya,Intro
2,I am Sundar and this is call from Google,Intro
3,I am calling about your Microsoft Azure subscr...,Purpose
4,This is a call regarding your Google Cloud Pla...,Purpose
5,I would like to talk about your Amazon Web Ser...,Purpose


In [53]:
# As machine only understands Number I convert Labels i.e intro and Purpose to 0 and 1 using label encoder

In [54]:
le= LabelEncoder()
df1["label"]=le.fit_transform(df1["Intent"])

In [55]:
df1

,text,Intent,label
0,My name is Jeff and I am calling from Amazon,Intro,0
1,I am calling from Microsoft and my name is Satya,Intro,0
2,I am Sundar and this is call from Google,Intro,0
3,I am calling about your Microsoft Azure subscr...,Purpose,1
4,This is a call regarding your Google Cloud Pla...,Purpose,1
5,I would like to talk about your Amazon Web Ser...,Purpose,1


In [56]:
####################################################################################################

In [57]:
# writing a function to do preprocessing on given text

In [58]:
def clean_text(data):
  remove_spaces= data.replace("\\n", "").replace("\t", "")
  remove_accented= unidecode(remove_spaces)
  mapping= fix(remove_accented)
  stopword_list= stopwords.words("english")
  stopword_list.remove("no")
  stopword_list.remove("nor")
  stopword_list.remove("not")
  tokens=word_tokenize(mapping)
  clean_data=[word.lower() for word in tokens if (word.lower() not in punctuation) and (word.lower() not in stopword_list) and ( len(word)>1) and (word.isalpha()) ]
  lemmatizer = WordNetLemmatizer()
  final_text = []
  for word in clean_data:
      lemmatized_word = lemmatizer.lemmatize(word)
      final_text.append(lemmatized_word)
  return " ".join(final_text)

In [59]:
# passing each record in tect to the function clean_text for preprocessing

In [60]:
df1['Clean_text']=df1['text'].apply(lambda x: clean_text(x))

In [61]:
df1

,text,Intent,label,Clean_text
0,My name is Jeff and I am calling from Amazon,Intro,0,name jeff calling amazon
1,I am calling from Microsoft and my name is Satya,Intro,0,calling microsoft name satya
2,I am Sundar and this is call from Google,Intro,0,sundar call google
3,I am calling about your Microsoft Azure subscr...,Purpose,1,calling microsoft azure subscription
4,This is a call regarding your Google Cloud Pla...,Purpose,1,call regarding google cloud platform account
5,I would like to talk about your Amazon Web Ser...,Purpose,1,would like talk amazon web service account


In [62]:
# keeping text in x ie independent variables and labels in y ie target

In [63]:
x=df1.text
y=df1.label

In [64]:
# Splitting the given data

In [65]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.33, stratify= y, random_state=2)

In [66]:
x_train.shape

(4,)

In [67]:
x_train

2             I am Sundar and this is call from Google
1     I am calling from Microsoft and my name is Satya
3    I am calling about your Microsoft Azure subscr...
5    I would like to talk about your Amazon Web Ser...
Name: text, dtype: object

In [68]:
####################################################################################################

In [69]:
# creating a counvectorizer model to convert words to numbers

In [70]:
cv= CountVectorizer()

In [71]:
cv.fit(x_train)

CountVectorizer()

In [72]:
x_train_cv=cv.transform(x_train)


In [73]:
#After transforming from Count Vectorizer we get sparse matrix , so we need to convert it to array so we add x_train.A instead of x_train

In [74]:
x_train_cv.A

array([[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 1]])

In [75]:
x_test=cv.transform(x_test)

In [76]:
####################################################################################################

In [77]:
# Creating a Random forest model for predicting intents for given text

In [78]:
rm_model= RandomForestClassifier()
rm_model.fit(x_train_cv.A,y_train)

RandomForestClassifier()

In [79]:
####################################################################################################

In [80]:
# passing the text for prediction of intent

In [81]:
t=clean_text("I am Palash and this is call from Dimensionless.")
t2=cv.transform([t])
prediction=rm_model.predict(t2.A)

In [82]:
if prediction[0]==0:
  intent="Intro"
else:
  intent="Purpose"

print(intent)

Purpose


In [83]:
# rm_model is predicting the intent correctly

In [84]:
####################################################################################################

In [85]:
# as we want output in json format as shown below

In [86]:
import json

In [87]:
# {
#   "task_1_output": text,
#     "task_3_output": [
#       {
#         "sentence": text,
#         "intent": "intro",
#         "entities": [
#           {
#             "entity_name": "caller_name",
#             "entity_value": PERSON
#           },
#           {
#             "entity_name": "company",
#             "entity_value": ORG
#           }
#         ]
#       }
#     ]
# }

In [88]:
# i created a dictionary d which will store entity labels as key and its text as value

In [89]:
d={}
# Create a document 
document = nlp(text)

# Entity text & label extraction
for entity in document.ents:
    d[entity.label_]=entity.text
    

In [90]:
d


{'PERSON': 'Mike', 'ORG': 'AT&T Incorporation'}

In [91]:
text

'hello Nancy this is Mike from AT&T Incorporation'

In [92]:
d['PERSON']

'Mike'

In [93]:
####################################################################################################

In [94]:
def output(texts, intents, dict1):
  # initializing dictionary
  # Initialize JSON data
  test_dict = {"task_1_output": "text", \
                "task_3_output": [{"sentence": "text","intent": "intro", "entities": [{"entity_name": "caller_name","entity_value": "P"}, \
                {"entity_name": "company","entity_value": "ORG"}]}]}

  test_dict["task_1_output"] = text
  test_dict["task_3_output"][0]["sentence"] = text
  test_dict["task_3_output"][0]["intent"] = intent
  test_dict["task_3_output"][0]["entities"][0]["entity_value"] = d["PERSON"]
  test_dict["task_3_output"][0]["entities"][1]["entity_value"] = d["ORG"]

  # printing original dictionary
  print("The original dictionary is : ", test_dict)
  
  # using json.dumps() to Pretty Print
  pretty_dict = json.dumps(test_dict, indent=4)
  print("The Pretty Print dictionary is : \n", pretty_dict)
  return

In [95]:
output(text, intent, d)

The original dictionary is :  {'task_1_output': 'hello Nancy this is Mike from AT&T Incorporation', 'task_3_output': [{'sentence': 'hello Nancy this is Mike from AT&T Incorporation', 'intent': 'Purpose', 'entities': [{'entity_name': 'caller_name', 'entity_value': 'Mike'}, {'entity_name': 'company', 'entity_value': 'AT&T Incorporation'}]}]}
The Pretty Print dictionary is : 
 {
    "task_1_output": "hello Nancy this is Mike from AT&T Incorporation",
    "task_3_output": [
        {
            "sentence": "hello Nancy this is Mike from AT&T Incorporation",
            "intent": "Purpose",
            "entities": [
                {
                    "entity_name": "caller_name",
                    "entity_value": "Mike"
                },
                {
                    "entity_name": "company",
                    "entity_value": "AT&T Incorporation"
                }
            ]
        }
    ]
}


In [96]:
# As Seen above I have correctly given the output of my assigment and if there is any othe audio you want to chech pass the audio to the below function

In [97]:
def find_intent_form_speech(audio):
  convert(audio)
  print('*'*50)
  print_entities(nlp, text)
  print('*'*50)
  visualize_entities(nlp, text)
  print('*'*50)
  t=clean_text(text)
  t2=cv.transform([t])
  prediction=rm_model.predict(t2.A)
  print('*'*50)
  d={}
  document = nlp(text)
  for entity in document.ents:
      d[entity.label_]=entity.text
  print('*'*50)
  aa=output(text, intent, d)
  return aa

In [98]:
find_intent_form_speech("/content/sales_call_telephone_marketers.wav")

hello Nancy this is Mike from AT&T Incorporation
**************************************************
Nancy-> PERSON
Mike-> PERSON
AT&T Incorporation-> ORG
**************************************************


**************************************************
**************************************************
**************************************************
The original dictionary is :  {'task_1_output': 'hello Nancy this is Mike from AT&T Incorporation', 'task_3_output': [{'sentence': 'hello Nancy this is Mike from AT&T Incorporation', 'intent': 'Purpose', 'entities': [{'entity_name': 'caller_name', 'entity_value': 'Mike'}, {'entity_name': 'company', 'entity_value': 'AT&T Incorporation'}]}]}
The Pretty Print dictionary is : 
 {
    "task_1_output": "hello Nancy this is Mike from AT&T Incorporation",
    "task_3_output": [
        {
            "sentence": "hello Nancy this is Mike from AT&T Incorporation",
            "intent": "Purpose",
            "entities": [
                {
                    "entity_name": "caller_name",
                    "entity_value": "Mike"
                },
                {
                    "entity_name": "company",
                    "entity_value

In [99]:
#Results obtained satisfy the given acessment problem statement